# ESN Experiment

This notebook reproduces the results in the paper ...

It consits of three parts:

1. An example for how to set up and experiment and use the code to train an ESN on the data.
2. A section to reproduce the scores mentioned in the paper for each testset.
3. A section to measure training time of an echo state network on our dataset.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import DataSet
import Evaluation
import datetime
import numpy as np 
import matplotlib.pyplot as plt
import random
from matplotlib.backends.backend_pdf import PdfPages

from Utils import *

from DataSet import UniHHIMUGestures
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
manualSeed = 1

np.random.seed(manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
# if you are suing GPU
torch.cuda.manual_seed(manualSeed)
torch.cuda.manual_seed_all(manualSeed)


#torch.backends.cudnn.enabled = False 
#torch.backends.cudnn.benchmark = False
#torch.backends.cudnn.deterministic = True

We need to specify a bunch of parameters the expriment:

In [ ]:
#===========================================================================
# Give this run a name. 
# If name equals 'test', no log will be generated
#===========================================================================
name = 'test'


#===========================================================================
# Decide which gesture data shall be used for training
#===========================================================================
inputGestures = [0,1,2,3,4,5,6,7,8,9]

#===========================================================================
# Decide which target signals shall be used for training
#===========================================================================
usedGestures = [0,1,2,3,4,5,6,7,8,9]

#===========================================================================
# Concatenate data to create "more" training samples, 1 corresponds to no concatenations
#===========================================================================
concFactor = 1

#===========================================================================
# Add noise to the data, 0 corresponds to no noise. Noise above 2 has shown to weaken recognition
#===========================================================================
noiseFactor = 1

#===========================================================================
# Decide wether gestures shall be shuffled before training. If true, nFolds many 
# pieces will be generated. Not every piece is garanteed to contain every gesture, so do not use too many.
#===========================================================================
shuffle = True
nFolds = 4


#===========================================================================
# Function used to evaluate during cross validation. Possible functions are:
# Evaluation.calc1MinusF1FromMaxApp (best working, used in thesis)
# Oger.utils.nmse (normalised mean square error, tells nothing about classifier perfomance but works okay)
# Evaluation.calcLevenshteinError (use the Levenshtein error, disadvantages are highlighted in thesis) 
# Evaluation.calc1MinusF1FromInputSegment (use segmentation by supervised signal)
#===========================================================================
evaluationFunction = Evaluation.calc1MinusF1FromMaxApp

#===========================================================================
# Set this to true if another output neuron shall be added to represent "no gesture"
#===========================================================================
learnTreshold = False

#===========================================================================
# Use on of the optimisation dictionaries from the optDicts file
#===========================================================================
optDict = 'bestParas'

#===========================================================================
# Use normalizer
#===========================================================================
useNormalized = 2

#===========================================================================
# Pick datasets to train on, and datasets to test on
#===========================================================================
inputFiles = ['stephan','julian','nadja','line']
testFiles = ['nike']

# If desired add a specific file to test on, e.g. randTestFiles = ['lana_0_0.npz']
randTestFiles = []



#===========================================================================
# Setup project directory
#===========================================================================
now = datetime.datetime.now()
resultsPath = getProjectPath()+'results/'
pdfFileName = now.strftime("%Y-%m-%d-%H-%M")+'_'+name+'.pdf'
pdfFilePath = resultsPath+'pdf/'+pdfFileName
npzFileName = now.strftime("%Y-%m-%d-%H-%M")+'_'+name+'.npz'
npzFilePath = resultsPath+'npz/'+npzFileName
bestFlowPath = resultsPath+'nodes/'+now.strftime("%Y-%m-%d-%H-%M")+'_'+name+'.p'
pp = PdfPages(pdfFilePath)


#===========================================================================
# Add labels for gestures
#===========================================================================
totalGestureNames = ['left','right','forward','backward','bounce up','bounce down','turn left','turn right','shake lr','shake ud', \
                     'tap 1','tap 2','tap 3','tap 4','tap 5','tap 6','no gesture']
gestureNames = []
for i in usedGestures:
    gestureNames.append(totalGestureNames[i])
gestureNames.append('no gesture')

Create dataset and dataloaders

In [ ]:
def createData(inputFiles, testFiles):
    trainset = UniHHIMUGestures(dataDir='dataSets/', 
                                train=True, 
                                inputFiles=inputFiles,
                                testFiles=testFiles,
                                useNormalized=useNormalized, 
                                learnTreshold=learnTreshold,
                                shuffle=True,
                               )

    testset = UniHHIMUGestures(dataDir='dataSets/', 
                               train=False, 
                               inputFiles=inputFiles,
                               testFiles=testFiles,
                               useNormalized=useNormalized, 
                               learnTreshold=learnTreshold,
                               shuffle=True
                              
                              )

    trainloader = DataLoader(trainset, batch_size=1,
                            shuffle=True, num_workers=1)
    testloader = DataLoader(testset, batch_size=1,
                            shuffle=True, num_workers=1)
    return trainset, testset, trainloader, testloader
    
trainset, testset, trainloader, testloader = createData(inputFiles, testFiles)

Let's take a look at the scaled input data:

In [ ]:
fig, ax = plt.subplots(3,1, figsize=(20,9))
ax[0].plot(trainset[0][0][:800,0:3])
ax[1].plot(trainset[0][0][:800,3:6])
ax[2].plot(trainset[0][0][:800,6:9])
plt.tight_layout()

Looks all good, we can clearly see the gesture sequences intercepted by non gesture seqeuences in between.
Now let's create a Leaky Integrator ESN with parameters as defined in the paper.

In [ ]:
from echotorch.nn.ESN import ESN
from echotorch.nn.LiESN import LiESN


def createESN():
    esn = LiESN(
        input_dim=9,
        hidden_dim=400,
        output_dim=10 if not learnTreshold else 11,
        input_scaling=13.,
        sparsity=0.1, # input matrix sparsity
        w_distrib='gaussian',
        spectral_radius=1.,
        bias_scaling=0., # no input bias
        feedbacks=False, # No feedback connections
        learning_algo='inv',
        leaky_rate=.3
    )
    return esn

esn = createESN()

## Training

Training is done automatically when presenting the values and targets to the network. After calling finalize training is completed and network switches to prediction mode.

In [ ]:
from torch.autograd import Variable

def trainESN(trainloader, esn):
    for inputs, targets in trainloader:
        inputs, targets = Variable(inputs.float()), Variable(targets.float())
        esn(inputs, targets)

    esn.finalize()


trainESN(trainloader, esn)

## Testing

Plot activations of network on testset and corresponding target signals.

In [ ]:
for test_inputs, test_targets in testloader:
    plt.figure(figsize=(20,5))
    outputs = esn(test_inputs.float())
    if learnTreshold:
        plt.plot(outputs[0,:800,10], c='black')
    plt.plot(outputs[0,:800,:10])
    plt.plot(test_targets[0,:800,:])
plt.tight_layout()

In [ ]:
import sklearn

def testESN(testloader, learnTreshold, fixed_threshold=0.4, plot=False):
    
    for test_inputs, test_targets in testloader:
        outputs = esn(test_inputs.float())
    
        testCms = []
        testF1MaxApps = []

        t_target = test_targets[0].numpy()
        prediction = outputs[0].numpy()[:,:10]
        if learnTreshold: # if threshold is learned, then it's the last collumn of the prediction
            threshold = outputs[0].numpy()[:,10]
        else: #else add a constant threshold
            threshold = np.ones((prediction.shape[0],1))*fixed_threshold

        t_maxApp_prediction = Evaluation.calcMaxActivityPrediction(prediction,t_target,threshold, 10)


        pred_MaxApp, targ_MaxApp = Evaluation.calcInputSegmentSeries(t_maxApp_prediction, t_target, 0.5)
        testF1MaxApps.append(np.mean(sklearn.metrics.f1_score(targ_MaxApp,pred_MaxApp,average=None)))

        if plot:
            plt.figure(figsize=(20,3))
            plt.plot(t_maxApp_prediction[:800])
            plt.plot(t_target[:800])


        conf = sklearn.metrics.confusion_matrix(targ_MaxApp, pred_MaxApp)
        testCms.append(conf)

        if plot:
            Evaluation.plot_confusion_matrix(testCms[0], gestureNames, 'test set')
            plt.tight_layout()
            plt.ylim(10.5,-0.5)

        print("Test f1 score for maxactivity algorithm on testsets {} is {:.2f} ".format(
            testFiles, np.mean(testF1MaxApps)))
    return testF1MaxApps

f1scores = testESN(testloader, learnTreshold)


In [ ]:
f1scores

## Section 2: Evaluate on different Test Sets

In [ ]:
files = ['stephan','julian','nadja','line','nike']

all_scores = []

for idx in range(5):
    scores = []
    
    # Shuffle testsets
    inputFiles = files[:idx] + files[idx+1:]
    testFiles = files[idx:idx+1]


    for _ in range(10):
        trainset, testset, trainloader, testloader = createData(inputFiles=inputFiles, testFiles=testFiles)
        esn = createESN()
        trainESN(trainloader, esn)
        
        if False:
            totalTrainInputData = []
            totalTrainTargetData = []
            for inputs, targets in trainloader:
                totalTrainInputData.append(inputs)
                totalTrainTargetData.append(targets)
            totalTrainInputData = torch.cat(totalTrainInputData,1)
            totalTrainTargetData = torch.cat(totalTrainTargetData,1)
            totalTrainInputData = torch.tensor(totalTrainInputData)
            totalTrainPrediction = esn(totalTrainInputData.float())

            totalTrainPrediction = totalTrainPrediction[0,:,:].numpy()
            totalTrainTargetData = totalTrainTargetData[0,:,:].numpy()
        
            tresholds, _, bestF1ScoreTreshold = Evaluation.calcTPFPForThresholds(totalTrainPrediction, totalTrainTargetData, 'Train Data Confusion - Target Treshold', False, plot=False)
        bestF1ScoreTreshold = 0.4
        # very good results when bestF1score is set to 0.4
        
        score = testESN(testloader, learnTreshold, fixed_threshold=bestF1ScoreTreshold)
        scores.extend(score)
        
    print('{}: avg f1 score: {:.2f}, ({:.2f}) '.format(testFiles[0], np.array(scores).mean(),np.array(scores).std()))
    all_scores.extend(scores)

In [ ]:
(np.mean([0.624 - 0.62, 0.791 - 0.79, 0.809 - 0.78, 0.831 - 0.76, 0.926 - 0.88]),
np.mean([0.624 - 0.60, 0.791 - 0.79, 0.809 - 0.80, 0.831 - 0.76, 0.926 - 0.86]),)


print("paper mean: {:.3f}".format(np.mean([0.624,0.791,0.809,0.831,0.926])))


print("opti. threshold mean: {:.3f}".format(np.mean([0.60,0.79,0.80,0.76,0.86])))
print("fixed threshold mean: {:.3f}".format(np.mean([0.62,0.79,0.78,0.76,0.88])))






Results in comparison:

| Test Person   | Paper   | Paper Repro   | Opt. Treshold | Fixed Treshold |
| ------------- |:-------:| :-----------: | :-----------: | :------------: |
| Julian        | 0.624   | 0.59 (0.04)   | 0.60 (0.04)   | 0.62 (0.04)    |
| Nike          | 0.791   | 0.74 (0.05)   | 0.79 (0.03)   | 0.79 (0.04)    |
| Stephan       | 0.809   | 0.77 (0.03)   | 0.80 (0.03)   | 0.78 (0.04)    |
| Nadja         | 0.831   | 0.79 (0.04)   | 0.76 (0.06)   | 0.76 (0.04)    |
| Line          | 0.926   | 0.86 (0.03)   | 0.86 (0.03)   | 0.88 (0.04)    |
| ------------- |---------| ------------- | ------------- | ---------------|
| Diff paper    |         |               | 0.034         | 0.030          |

#### TODO: why is the testscore here lower than in OGER? supposed to be 0.8

## Section 3: Meassure training time

We are using timeit to meassure training time, two repetitions of 10 loops.

In [ ]:
print("Training on: {}".format(inputFiles))

In [ ]:
%%timeit -r2 -n10 
esn = createESN()
trainESN(trainloader, esn)

# Experimental code

Code below here is experimental and not used in the paper

#### Try out softmax

Learn threshold must be true for softmax to make sense.

In [ ]:
assert learnTreshold==True, 'Model must be created with learnThreshold=true to use softmax'

for test_inputs, test_targets in testloader:
    plt.figure(figsize=(20,5))
    outputs = esn(test_inputs.float())
    outputs = nn.Softmax(2)(outputs)
    if learnTreshold:
        plt.plot(outputs[0,:800,10], c='black')
    plt.plot(outputs[0,:800,:10])
    plt.plot(test_targets[0,:800,:])
plt.tight_layout()

In [ ]:

def calcBasicMaxActivation(prediction, t_target, threshold, gestureMinLength=1 ):
    inactive_timesteps = prediction.max(1) < threshold
    active_timesteps = prediction.max(1) >= threshold

    gesture_end = inactive_timesteps[1:] * active_timesteps[:-1]
    gesture_start = inactive_timesteps[:-1] * active_timesteps[1:]

    predicted_labels = np.zeros(prediction.shape)
    for start, end in list(zip(np.where(gesture_start)[0], np.where(gesture_end)[0])):

        if end-start > gestureMinLength:
            gestureclass = prediction[start:end].sum(0).argmax()
            predicted_labels[start:end,gestureclass] = 1
    return predicted_labels


In [ ]:
import sklearn
testCms = []
testF1MaxApps = []

t_target =  test_targets[0].numpy()
prediction = outputs[0].numpy()[:,:10]
if learnTreshold: # if threshold is learned, then it's the las collumn of the prediction
    threshold = outputs[0].numpy()[:,10]
else: #else add a constant threshold
    threshold = np.ones((prediction.shape[0],1))*0.4

# use different method for maxappactivation
t_maxApp_prediction = calcBasicMaxActivation(prediction,t_target,threshold, 10)


pred_MaxApp, targ_MaxApp = Evaluation.calcInputSegmentSeries(t_maxApp_prediction, t_target, 0.5)
testF1MaxApps.append(np.mean(sklearn.metrics.f1_score(targ_MaxApp,pred_MaxApp,average=None)))


#print(t_maxApp_prediction.shape, prediction.shape, pred_MaxApp, targ_MaxApp)
plt.figure(figsize=(20,3))
plt.plot(t_maxApp_prediction[:800])
plt.plot(t_target[:800])


conf = sklearn.metrics.confusion_matrix(targ_MaxApp, pred_MaxApp)
testCms.append(conf)

Evaluation.plot_confusion_matrix(testCms[0], gestureNames, 'test set')
plt.tight_layout()
plt.ylim(10.5,-0.5)

print("Test f1 score for maxactivity: {:.2f}, reg: {:.2f}".format(testF1MaxApps[0], trainF1s[0]))


